<a href="https://colab.research.google.com/github/jcs-lambda/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
#@title pre-done prep
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
#@title pre-done get/prep data
import numpy as np
import pandas as pd
import pandas_profiling

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [0]:
import re

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

!pip install arcgis 2>/dev/null >/dev/null

from arcgis.gis import GIS
from arcgis.geocoding import reverse_geocode

gis=GIS()

# Explore the data

In [4]:
df

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49347,1.0,2,2016-06-02 05:41:05,"30TH/3RD, MASSIVE CONV 2BR IN LUXURY FULL SERV...",E 30 St,40.7426,-73.9790,3200,230 E 30 St,medium,1,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
49348,1.0,1,2016-04-04 18:22:34,"HIGH END condo finishes, swimming pool, and ki...",Rector Pl,40.7102,-74.0163,3950,225 Rector Place,low,1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1
49349,1.0,1,2016-04-16 02:13:40,Large Renovated One Bedroom Apartment with Sta...,West 45th Street,40.7601,-73.9900,2595,341 West 45th Street,low,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
49350,1.0,0,2016-04-08 02:13:33,Stylishly sleek studio apartment with unsurpas...,Wall Street,40.7066,-74.0101,3350,37 Wall Street,low,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
df.dtypes

bathrooms               float64
bedrooms                  int64
created                  object
description              object
display_address          object
latitude                float64
longitude               float64
price                     int64
street_address           object
interest_level           object
elevator                  int64
cats_allowed              int64
hardwood_floors           int64
dogs_allowed              int64
doorman                   int64
dishwasher                int64
no_fee                    int64
laundry_in_building       int64
fitness_center            int64
pre-war                   int64
laundry_in_unit           int64
roof_deck                 int64
outdoor_space             int64
dining_room               int64
high_speed_internet       int64
balcony                   int64
swimming_pool             int64
new_construction          int64
terrace                   int64
exclusive                 int64
loft                      int64
garden_p

In [6]:
df.isnull().sum()

bathrooms                  0
bedrooms                   0
created                    0
description             1425
display_address          133
latitude                   0
longitude                  0
price                      0
street_address            10
interest_level             0
elevator                   0
cats_allowed               0
hardwood_floors            0
dogs_allowed               0
doorman                    0
dishwasher                 0
no_fee                     0
laundry_in_building        0
fitness_center             0
pre-war                    0
laundry_in_unit            0
roof_deck                  0
outdoor_space              0
dining_room                0
high_speed_internet        0
balcony                    0
swimming_pool              0
new_construction           0
terrace                    0
exclusive                  0
loft                       0
garden_patio               0
wheelchair_access          0
common_outdoor_space       0
dtype: int64

In [7]:
df.describe()

,bathrooms,bedrooms,latitude,longitude,price,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
count,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000
mean,1.201794,1.537149,40.750760,-73.972760,3579.585247,0.524838,0.478276,0.478276,0.447631,0.424852,0.415081,0.367085,0.052769,0.268452,0.185653,0.175902,0.132761,0.138394,0.102833,0.087203,0.060471,0.055206,0.051908,0.046193,0.043305,0.042711,0.039331,0.027224,0.026241
std,0.470711,1.106087,0.038954,0.028883,1762.430772,0.499388,0.499533,0.499533,0.497255,0.494326,0.492741,0.482015,0.223573,0.443158,0.388830,0.380741,0.339320,0.345317,0.303744,0.282136,0.238359,0.228385,0.221844,0.209905,0.203544,0.202206,0.194382,0.162738,0.159852
min,0.000000,0.000000,40.575700,-74.087300,1375.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,40.728300,-73.991800,2500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,40.751700,-73.978000,3150.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,2.000000,40.774000,-73.955000,4095.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,10.000000,8.000000,40.989400,-73.700100,15500.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
df.describe(exclude='number')

,created,description,display_address,street_address,interest_level
count,48817,47392,48684,48807,48817
unique,48148,37853,8674,15135,3
top,2016-06-25 01:30:16,,Broadway,3333 Broadway,low
freq,3,1627,435,174,33946


# Clean the data

## remove null values
* description
* display address
* street address

In [9]:
#@title description
# replace NaNs with empty strings
df['description'] = df['description'].fillna('')
df['description'].isnull().any()

False

In [10]:
#@title street address
# replace NaNs using reverse geocoding with given latitude and longitude
# https://developers.arcgis.com/python/guide/reverse-geocoding/

def lat_long_to_address(row):
  latitude = row['latitude']
  longitude = row['longitude']
  street_address = reverse_geocode([longitude, latitude]).get('address').get('ShortLabel')
  row['street_address'] = street_address
  return row

df[df['street_address'].isna()] = df[df['street_address'].isna()].apply(lat_long_to_address, axis=1)
df['street_address'].isnull().any()

False

In [11]:
#@title display address
# replace NaNs with the street name (street address with leading numbers removed)

def street_address_to_display_address(row):
  row['display_address'] = re.sub('^[0-9]+ ', '', row['street_address'])
  return row

df[df['display_address'].isna()] = df[df['display_address'].isna()].apply(street_address_to_display_address, axis=1)
df['display_address'].isnull().any()

False

## created column to datetime

In [12]:
df['created'] = pd.to_datetime(df['created'], infer_datetime_format=True, errors='coerce')
df['created'].dtype.name

'datetime64[ns]'

# Engineer at least 2 new features

In [0]:
#@title add total_perks column

perk_columns = []
for column in df.columns:
  if df[column].dtype == int and column != 'bedrooms' and column != 'price':
    perk_columns.append(column)

df['total_perks'] = df[perk_columns].sum(axis=1)

In [0]:
#@title add excitedness column
# calculate 'excitedness' as the number of exclamation points in the description
# weighted by interest_level
# low interest = 1/3, medium = 2/3, high = 1

interest_level_weights = {
    'low' : 1 / 3,
    'medium' : 2 / 3,
    'high' : 1
}

def calculate_excitedness(row):
  return row['description'].count('1') * interest_level_weights.get(row['interest_level'])

df['excitedness'] = df.apply(calculate_excitedness, axis=1)

# Do test/train split
* Use data from April & May 2016 to train.
* Use data from June 2016 to test.

In [15]:
split_date = pd.datetime(2016, 6, 1)

df_train = df[df['created'] < split_date]
df_test = df[df['created'] >= split_date]

print(f'Train created min: {min(df_train["created"])}; max: {max(df_train["created"])}')
print(f'Test  created min: {min(df_test["created"])}; max: {max(df_test["created"])}')

Train created min: 2016-04-01 22:12:41; max: 2016-05-31 23:10:48
Test  created min: 2016-06-01 01:10:37; max: 2016-06-29 21:41:47


# Fit a linear regression model with at least 2 features

# Get regression metrics RMSE, MAE, and  𝑅2 , for both the train and test 

# Get the model's coefficients and intercept.

In [0]:
def linear_regression_stats(df_train, df_test, target, features):
  # 2. instantiate the class
  model = LinearRegression()
  
  # 3. Arrange X features matrices and y target vector
  X_train = df_train[features]
  X_test = df_test[features]
  y_train = df_train[target]
  y_test = df_test[target]
  print(f'Linear Regression, targeting {target}, dependent on: {features}')

  print()

  # 4. Fit the model
  model.fit(X_train, y_train)
  y_pred = model.predict(X_train)
  rmse = np.sqrt(mean_squared_error(y_train, y_pred))
  mae = mean_absolute_error(y_train, y_pred)
  r2 = r2_score(y_train, y_pred)
  print(f'Train root mean squared error: ${rmse:,.2f}')
  print(f'Train mean absolute error: ${mae:,.2f}')
  print(f'Train R^2: {r2 * 100:,.2f}%')
  
  print() 

  # 5. Apply the model to new data
  y_pred = model.predict(X_test)
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))
  mae = mean_absolute_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)
  print(f'Test root mean squared error: ${rmse:,.2f}')
  print(f'Test mean absolute error: ${mae:,.2f}')
  print(f'Test R^2: {r2 * 100:,.2f}%')

  print()

  model_intercept_and_coefficients = {
      'Intercept' : model.intercept_,
      **pd.Series(model.coef_, features).to_dict()
  }

  return model_intercept_and_coefficients

In [17]:
features = ['bedrooms', 'bathrooms', 'total_perks', 'excitedness', 'latitude', 'longitude']
target = 'price'

linear_regression_stats(df_train, df_test, target, features)

Linear Regression, targeting price, dependent on: ['bedrooms', 'bathrooms', 'total_perks', 'excitedness', 'latitude', 'longitude']

Train root mean squared error: $1,132.23
Train mean absolute error: $725.99
Train R^2: 58.71%

Test root mean squared error: $1,117.60
Test mean absolute error: $730.45
Test R^2: 59.81%



{'Intercept': -1166470.7323329316,
 'bathrooms': 1914.4846488283338,
 'bedrooms': 431.33824140822594,
 'excitedness': -103.06139342934401,
 'latitude': 1962.7450136270015,
 'longitude': -14693.631667297788,
 'total_perks': 50.51748189096156}